#  <p style="color:blue">Real Estates in France 1st semester 2020</p>


In [76]:
import pandas as pd
import matplotlib as plt

### Connexion to Database (try with 2 differents connectors)

##### With mysql connector

In [77]:
import mysql.connector as sql

In [78]:
sql_=sql.connect(host = "localhost",
    user = "Phil",
    password = "")

In [79]:
cursor=sql_.cursor()

In [80]:
cursor.execute('show databases;')

In [81]:
for i in cursor:
    print(i)

('dataimmo',)
('foodly',)
('information_schema',)
('moviz',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [82]:
sql_.close

<bound method CMySQLConnection.close of <mysql.connector.connection_cext.CMySQLConnection object at 0x000001BCFD212160>>

#### With sqlalceimy

In [83]:
import sqlalchemy as sql_al

In [84]:
engine=sql_al.create_engine('mysql://Phil:@127.0.0.1/dataimmo')

In [85]:
tables=engine.execute('show tables')

In [86]:
for i in tables:
    print(i)

('adresse',)
('bienimmo',)
('temp_adresse',)
('vente',)


## Try of different queries

In [87]:
query = "select * from bienimmo limit 10"
df = pd.read_sql_query(query,engine)

In [88]:
df

,id,numLot,surfLot,nbrePieces,typLoc,adresse_id
0,1,22,50.42,2,Appartement,1
1,2,12,48.22,3,Appartement,2
2,3,146,130.80,6,Appartement,3
3,4,11,109.22,5,Maison,4
4,5,31,108.65,4,Appartement,5
5,6,50,31.65,2,Appartement,6
6,7,11,52.58,2,Appartement,7
7,8,304,58.71,2,Appartement,8
8,9,14,93.23,4,Maison,9
9,10,21,117.00,5,Maison,10


##### Proportion of sales for each type of flat (Studio (1), 1 bedroom (2), etc...)

In [89]:
query="select nbrePieces,(count(*)/31377)*100 as ProportionVentes  from bienimmo join vente on bienimmo.id=vente.bien_id where bienimmo.typLoc='Appartement' group by nbrePieces order by ProportionVentes desc "

In [90]:
df_ventes_pieces = pd.read_sql_query(query,engine)
df_ventes_pieces

,nbrePieces,ProportionVentes
0,2,31.1789
1,3,28.5719
2,1,21.4775
3,4,14.2142
4,5,3.5504
5,6,0.6502
6,7,0.1721
7,0,0.0956
8,8,0.0542
9,9,0.0255


##### Price per m² of flats  per departements ordered from the highest to the lowest

In [91]:
query="Select codeDep,format ((avg(valFonc)/avg(surfLot)),2) as prixM2 from vente join bienImmo on vente.bien_id=bienImmo.id join adresse on bienImmo.adresse_id=adresse.id where bienImmo.typLoc='Appartement' group by codeDep order by avg(valFonc) desc limit 10;"


In [92]:
m2Price_per_departement= pd.read_sql_query(query,engine)
m2Price_per_departement

,codeDep,prixM2
0,75,"11,901.66"
1,92,"7,218.90"
2,94,"4,931.94"
3,69,"3,915.43"
4,6,"4,611.13"
5,78,"4,015.46"
6,74,"4,108.95"
7,93,"4,078.67"
8,20,"3,543.43"
9,1,"2,828.37"


In [93]:
query='DESCRIBE adresse'

In [94]:
adresse=pd.read_sql_query(query,engine)
adresse

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,numVoie,int,YES,,None,
2,typVoie,varchar(5),YES,,None,
3,voie,varchar(30),YES,,None,
4,cp,int,YES,,None,
5,ville,varchar(30),YES,,None,
6,codeDep,int,YES,,None,


##### Evolution rate from first to second trimester 

In [95]:
query="with TR1 as (select count(*) as NbreVentes from vente where datMut>='2020-01-01' and datMut<='2020-03-31'),\
     TR2 as (select count(*) as NbreVentes from vente where  datMut>='2020-04-01' and  datMut<='2020-06-30')\
     select ((TR2.NbreVentes-TR1.NbreVentes)/TR1.NbreVentes)*100 as SalesVariationFromTr1ToTr2 from TR1 join TR2;"  

In [96]:
evolOfSales=pd.read_sql_query(query,engine)
evolOfSales

,SalesVariationFromTr1ToTr2
0,3.6719


##### Liste of cities where the sales rate has evolved by 20% and more

In [97]:
P1="with TR1 as (select ville, count(*) as NbreVentes from vente join bienimmo on vente.bien_id=bienimmo.id join "
P2="adresse on bienimmo.adresse_id=adresse.id where datMut>='2020-01-01' and datMut<='2020-03-31' group by ville), "
P3="TR2 as (select ville, count(*) as NbreVentes from vente join bienimmo on vente.bien_id=bienimmo.id join adresse "
P4="on bienimmo.adresse_id=adresse.id where datMut>='2020-04-01' and datMut<='2020-06-30' group by ville) "
P5="select TR2.ville,TR1.NbreVentes as NbreVenteTR1,TR2.NbreVentes as NbreVenteTR2, "
P6="format(((TR2.NbreVentes-TR1.NbreVentes)/TR1.NbreVentes)*100 ,0) as PourcentDiff "
P7="from TR2 join TR1 on TR2.ville=TR1.ville where TR2.NbreVentes>=1.2*TR1.NbreVentes group by TR2.ville limit 25;"
query=P1+P2+P3+P4+P5+P6+P7

In [98]:
citiesList_20=pd.read_sql_query(query,engine)
citiesList_20

,ville,NbreVenteTR1,NbreVenteTR2,PourcentDiff
0,DIVONNE-LES-BAINS,5,6,20
1,LAON,11,14,27
2,VILLERS-COTTERETS,3,5,67
3,CHATEAU-ARNOUX-SAINT-AUBAN,1,2,100
4,BARCELONNETTE,2,5,150
5,SAINT-MARTIN-DE-BROMES,1,2,100
6,BRIANCON,5,6,20
7,ORCIERES,1,5,400
8,GAP,2,6,200
9,EMBRUN,1,2,100


#### Top 3 cities indepartement:6,13,33,59,69

In [99]:
query="with Clt as (select ville,codeDep as Département,  format(avg(valFonc),0) as MoyFoncière,rank() over (partition by codeDep order by avg(valFonc) desc)  as classement from vente join bienImmo on vente.bien_id=bienImmo.id join adresse on bienImmo.adresse_id=adresse.id where codeDep=6 or codeDep=13 or codeDep=33 or codeDep=59 or codeDep=69 group by ville) select * from Clt where classement<4;"

In [100]:
Top3=pd.read_sql_query(query,engine)
Top3

,ville,Département,MoyFoncière,classement
0,SAINT-JEAN-CAP-FERRAT,6,"968,750",1
1,EZE,6,"655,000",2
2,MOUANS-SARTOUX,6,"476,898",3
3,GIGNAC-LA-NERTHE,13,"330,000",1
4,SAINT SAVOURNIN,13,"314,425",2
5,CASSIS,13,"313,417",3
6,LEGE-CAP-FERRET,33,"549,501",1
7,VAYRES,33,"335,000",2
8,ARCACHON,33,"307,436",3
9,BERSEE,59,"433,202",1


## Descriptive analysis

#### Getting the tables and join them

In [101]:
bienImmo=pd.read_sql_table("bienimmo",engine)
vente=pd.read_sql_table("vente",engine)
adress=pd.read_sql_table("adresse",engine)

In [102]:
bienImmo.head()

,id,numLot,surfLot,nbrePieces,typLoc,adresse_id
0,1,22,50.42,2,Appartement,1
1,2,12,48.22,3,Appartement,2
2,3,146,130.80,6,Appartement,3
3,4,11,109.22,5,Maison,4
4,5,31,108.65,4,Appartement,5


In [103]:
adress.head()

,id,numVoie,typVoie,voie,cp,ville,codeDep
0,1,190,RUE,CENTRALE,1370,SAINT-ETIENNE-DU-BOIS,1
1,2,347,RUE,DU CHATEAU,1170,CHEVRY,1
2,3,58,AV,DU MONT BLANC,1220,DIVONNE-LES-BAINS,1
3,4,140,RUE,DE L'ABBE JOLIVET,1630,PERON,1
4,5,39,RUE,BUFFON,1200,VALSERHONE,1


In [104]:
vente.head()

,id,datMut,valFonc,natMut,bien_id
0,1,2020-02-03,56000.0,Vente,1
1,2,2020-01-02,165000.0,Vente,2
2,3,2020-01-08,720000.0,Vente,3
3,4,2020-01-06,429250.0,Vente,4
4,5,2020-01-07,220900.0,Vente,5


In [105]:
df_bienImmo=pd.merge(bienImmo,adress,left_on="adresse_id",right_on="id")

In [106]:
df_bienImmo

,id_x,numLot,surfLot,nbrePieces,typLoc,adresse_id,id_y,numVoie,typVoie,voie,cp,ville,codeDep
0,1,22,50.42,2,Appartement,1,1,190,RUE,CENTRALE,1370,SAINT-ETIENNE-DU-BOIS,1
1,2,12,48.22,3,Appartement,2,2,347,RUE,DU CHATEAU,1170,CHEVRY,1
2,3,146,130.80,6,Appartement,3,3,58,AV,DU MONT BLANC,1220,DIVONNE-LES-BAINS,1
3,4,11,109.22,5,Maison,4,4,140,RUE,DE L'ABBE JOLIVET,1630,PERON,1
4,5,31,108.65,4,Appartement,5,5,39,RUE,BUFFON,1200,VALSERHONE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34163,34164,113,102.09,3,Appartement,34164,34164,4,RUE,DE RIVOLI,75004,PARIS 04,75
34164,34165,67,55.95,3,Appartement,34165,34165,38,RUE,DU TEMPLE,75004,PARIS 04,75
34165,34166,22,32.86,2,Appartement,34166,34166,13,RUE,DES TOURNELLES,75004,PARIS 04,75
34166,34167,89,56.80,3,Appartement,34167,34167,16,RUE,CHARLEMAGNE,75004,PARIS 04,75


In [107]:
df_bienImmo=pd.merge(df_bienImmo,vente,left_on="id_x",right_on="bien_id")

In [108]:
df_bienImmo

,id_x,numLot,surfLot,nbrePieces,typLoc,adresse_id,id_y,numVoie,typVoie,voie,cp,ville,codeDep,id,datMut,valFonc,natMut,bien_id
0,1,22,50.42,2,Appartement,1,1,190,RUE,CENTRALE,1370,SAINT-ETIENNE-DU-BOIS,1,1,2020-02-03,56000.0,Vente,1
1,2,12,48.22,3,Appartement,2,2,347,RUE,DU CHATEAU,1170,CHEVRY,1,2,2020-01-02,165000.0,Vente,2
2,3,146,130.80,6,Appartement,3,3,58,AV,DU MONT BLANC,1220,DIVONNE-LES-BAINS,1,3,2020-01-08,720000.0,Vente,3
3,4,11,109.22,5,Maison,4,4,140,RUE,DE L'ABBE JOLIVET,1630,PERON,1,4,2020-01-06,429250.0,Vente,4
4,5,31,108.65,4,Appartement,5,5,39,RUE,BUFFON,1200,VALSERHONE,1,5,2020-01-07,220900.0,Vente,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34163,34164,113,102.09,3,Appartement,34164,34164,4,RUE,DE RIVOLI,75004,PARIS 04,75,34164,2020-06-30,1290000.0,Vente,34164
34164,34165,67,55.95,3,Appartement,34165,34165,38,RUE,DU TEMPLE,75004,PARIS 04,75,34165,2020-06-30,726000.0,Vente,34165
34165,34166,22,32.86,2,Appartement,34166,34166,13,RUE,DES TOURNELLES,75004,PARIS 04,75,34166,2020-06-25,420000.0,Vente,34166
34166,34167,89,56.80,3,Appartement,34167,34167,16,RUE,CHARLEMAGNE,75004,PARIS 04,75,34167,2020-06-25,725000.0,Vente,34167


In [109]:
df_bienImmo=df_bienImmo.drop(["id_x","id_y","adresse_id","id","bien_id"],axis=1)

In [110]:
df_bienImmo

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
0,22,50.42,2,Appartement,190,RUE,CENTRALE,1370,SAINT-ETIENNE-DU-BOIS,1,2020-02-03,56000.0,Vente
1,12,48.22,3,Appartement,347,RUE,DU CHATEAU,1170,CHEVRY,1,2020-01-02,165000.0,Vente
2,146,130.80,6,Appartement,58,AV,DU MONT BLANC,1220,DIVONNE-LES-BAINS,1,2020-01-08,720000.0,Vente
3,11,109.22,5,Maison,140,RUE,DE L'ABBE JOLIVET,1630,PERON,1,2020-01-06,429250.0,Vente
4,31,108.65,4,Appartement,39,RUE,BUFFON,1200,VALSERHONE,1,2020-01-07,220900.0,Vente
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34163,113,102.09,3,Appartement,4,RUE,DE RIVOLI,75004,PARIS 04,75,2020-06-30,1290000.0,Vente
34164,67,55.95,3,Appartement,38,RUE,DU TEMPLE,75004,PARIS 04,75,2020-06-30,726000.0,Vente
34165,22,32.86,2,Appartement,13,RUE,DES TOURNELLES,75004,PARIS 04,75,2020-06-25,420000.0,Vente
34166,89,56.80,3,Appartement,16,RUE,CHARLEMAGNE,75004,PARIS 04,75,2020-06-25,725000.0,Vente


#### Description

In [111]:
df_bienImmo.describe()

,surfLot,nbrePieces,numVoie,cp,codeDep,valFonc
count,34168.000000,34168.000000,34168.000000,34168.000000,34168.000000,3.415000e+04
mean,57.644357,2.616015,448.743825,62089.903243,66.837304,2.528493e+05
std,53.336378,1.222276,1709.768481,28259.658901,74.174933,3.252639e+05
min,0.400000,0.000000,0.000000,1090.000000,1.000000,5.375000e+02
25%,34.600000,2.000000,8.000000,38000.000000,38.000000,1.040000e+05
50%,53.000000,3.000000,22.000000,75008.000000,75.000000,1.690000e+05
75%,72.310000,3.000000,65.000000,83622.500000,83.000000,2.850000e+05
max,5153.000000,11.000000,9999.000000,97490.000000,974.000000,9.000000e+06


Type for each columns

In [112]:
df_bienImmo.dtypes

numLot                object
surfLot              float64
nbrePieces             int64
typLoc                object
numVoie                int64
typVoie               object
voie                  object
cp                     int64
ville                 object
codeDep                int64
datMut        datetime64[ns]
valFonc              float64
natMut                object
dtype: object

#### Missing values

In [113]:
df_bienImmo.isnull().sum()

numLot         0
surfLot        0
nbrePieces     0
typLoc         0
numVoie        0
typVoie        0
voie           0
cp             0
ville          0
codeDep        0
datMut         0
valFonc       18
natMut         0
dtype: int64

In [114]:
df_bienImmo[df_bienImmo['valFonc'].isna()]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
456,487,17.34,1,Appartement,5360,ESP,DESPIERRE,5560,VARS,5,2020-02-12,NaN,Vente
1569,3,52.02,2,Appartement,25,RUE,DES PARAMIDEAUX,6110,LE CANNET,6,2020-05-04,NaN,Vente
3871,58,85.22,5,Maison,155,RUE,DES FAUVETTES,16600,RUELLE-SUR-TOUVRE,16,2020-06-05,NaN,Vente
4080,5,40.62,2,Appartement,4,BD,MICHELET,19100,BRIVE LA GAILLARDE,19,2020-06-25,NaN,Vente
4867,2,26.81,2,Appartement,30,RUE,CHAUVEAU LAGARDE,28000,CHARTRES,28,2020-06-20,NaN,Vente
5281,2,45.64,2,Appartement,5421,RES,DU PARC 3,20167,SARROLA-CARCOPINO,20,2020-05-30,NaN,Vente
10216,5,93.92,5,Maison,7,RUE,ALFRED SISLEY,44800,ST-HERBLAIN,44,2020-06-12,NaN,Vente
13563,18,35.47,2,Appartement,74,RUE,ARTHUR LAMENDIN,62400,BETHUNE,62,2020-02-26,NaN,Vente
13592,12,40.02,2,Appartement,13,RUE,DE LA PORTE NEUVE,62200,BOULOGNE SUR MER,62,2020-01-30,NaN,Vente
16369,2,34.64,2,Appartement,15,QUAI,DE LA SOMME,76200,DIEPPE,76,2020-01-23,NaN,Vente


#### Replacing missing values

In [115]:
#function to replace
def replace_valFonc(idrow,city):
    mis_val=df_bienImmo[df_bienImmo['ville']==city].valFonc.mean()
    df_bienImmo.loc[idrow,'valFonc']=mis_val

In [116]:
replace_valFonc(456,'VARS')

In [117]:
#Verication for the 1st one
df_bienImmo.loc[456]

numLot                        487
surfLot                     17.34
nbrePieces                      1
typLoc                Appartement
numVoie                      5360
typVoie                       ESP
voie                    DESPIERRE
cp                           5560
ville                        VARS
codeDep                         5
datMut        2020-02-12 00:00:00
valFonc             139666.666667
natMut                      Vente
Name: 456, dtype: object

In [118]:
replace_valFonc(1569,'LE CANNET')

In [119]:
replace_valFonc(3871,'RUELLE-SUR-TOUVRE')

In [120]:
replace_valFonc(4080,"BRIVE LA GAILLARDE")

In [121]:
replace_valFonc(4867,"CHARTRES")

In [122]:
replace_valFonc(5281,"SARROLA-CARCOPINO")

In [123]:
replace_valFonc(10216,"ST-HERBLAIN")

In [124]:
replace_valFonc(13563,"BETHUNE")

In [125]:
replace_valFonc(13592,"BOULOGNE SUR MER")

In [126]:
replace_valFonc(16369,"DIEPPE")

In [127]:
replace_valFonc(20794,"SAINT-CYR-SUR-MER")

In [128]:
replace_valFonc(21004,"L ISLE SUR LA SORGUE")

In [129]:
replace_valFonc(25451,"VILLEPINTE")

In [130]:
replace_valFonc(25530,"SEVRAN")

In [131]:
replace_valFonc(27712,"NOGENT-SUR-MARNE")

In [132]:
replace_valFonc(29370,"PARIS 01")

In [133]:
replace_valFonc(31114,"PARIS 11")

In [134]:
replace_valFonc(33934,"PARIS 04")

#### Vérification

In [135]:
df_bienImmo.isnull().sum()

numLot        0
surfLot       0
nbrePieces    0
typLoc        0
numVoie       0
typVoie       0
voie          0
cp            0
ville         0
codeDep       0
datMut        0
valFonc       0
natMut        0
dtype: int64

#### Duplicate

In [136]:
df_bienImmo.duplicated().sum()

0

#### Description of surLot column (surfaces of estates)

In [137]:
df_bienImmo["surfLot"].describe()

count    34168.000000
mean        57.644357
std         53.336378
min          0.400000
25%         34.600000
50%         53.000000
75%         72.310000
max       5153.000000
Name: surfLot, dtype: float64

Let see the estates with more than 600m²

In [138]:
df_bienImmo[df_bienImmo["surfLot"]>600]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
2621,2,1291.44,3,Appartement,23,BD,GAY LUSSAC,13014,MARSEILLE 14EME,13,2020-01-16,700000.0,Vente
3977,77,815.00,3,Appartement,34,CRS,VICTOR HUGO,17600,SAUJON,17,2020-03-11,154000.0,Vente
5420,5,742.17,2,Maison,5229,RUE,DE LA PRIVADIERE,30190,GARRIGUES SAINTE EULALIE,30,2020-02-27,1683000.0,Vente
8134,4,4936.00,3,Appartement,2,RUE,DES 6 FR RUELLAN,35400,SAINT-MALO,35,2020-02-20,223645.0,Vente
16690,1,1483.78,2,Appartement,12,RUE,DU PARC DES SPORTS,77170,BRIE COMTE ROBERT,77,2020-02-07,4098420.0,Vente
21856,2,5153.00,3,Appartement,90,RUE,DE LA JUSTICE,91230,MONTGERON,91,2020-02-05,265000.0,Vente
27143,1,2910.92,1,Appartement,3,ALL,DES COQUELICOTS,94470,BOISSY-SAINT-LEGER,94,2020-02-21,2500000.0,Vente


##### Cleaning this column

In [139]:
#Deleting the ones that seems the most improbable:
df_bienImmo=df_bienImmo.drop([2621,8134,16690,21856,27143,3977],axis=0)

#### Description of nbrePieces (number of rooms)

In [140]:
df_bienImmo["nbrePieces"].describe()

count    34162.000000
mean         2.616035
std          1.222341
min          0.000000
25%          2.000000
50%          3.000000
75%          3.000000
max         11.000000
Name: nbrePieces, dtype: float64

In [141]:
df_bienImmo[df_bienImmo["nbrePieces"]==0]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
153,420,2.36,0,Appartement,16,AV,DU JURA,1210,FERNEY-VOLTAIRE,1,2020-05-19,116500.0,Vente
1059,2,24.74,0,Appartement,1,MTE,DES GERANIUMS,6240,BEAUSOLEIL,6,2020-04-30,125000.0,Vente
1399,10,19.30,0,Appartement,51,AV,DES FRERES ROUSTAN,6220,VALLAURIS,6,2020-02-06,76500.0,Vente
2001,2,8.50,0,Appartement,17,RUE,DE LA FAUSSE PORTE,8600,GIVET,8,2020-01-30,91000.0,Vente
2606,26,33.46,0,Appartement,77,RUE,PAUTRIER,13004,MARSEILLE 4EME,13,2020-01-07,12000.0,Vente
3264,7,41.78,0,Appartement,122,RUE,LIANDIER,13008,MARSEILLE 8EME,13,2020-03-16,89000.0,Vente
8424,8,18.00,0,Appartement,22,CHE,JESUS,38000,GRENOBLE,38,2020-01-07,80750.0,Vente
8472,9,15.05,0,Appartement,11,RUE,YVES FARGE,38100,GRENOBLE,38,2020-01-27,61000.0,Vente
8624,14,27.42,0,Appartement,7,PL,DE LA REPUBLIQUE,38610,GIERES,38,2020-05-28,88000.0,Vente
10611,2,27.90,0,Appartement,26,AV,CHARLOTTE,44380,PORNICHET,44,2020-06-10,81859.0,Vente


This must be for very little studio.

In [142]:
df_bienImmo[df_bienImmo["nbrePieces"]==11]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
16300,205,50.91,11,Appartement,7,RUE,SAINT ELOI,76000,ROUEN,76,2020-05-27,139000.0,Vente


This one is probably 1 or 2 rooms and not 11 

Let see the average of surface for both

In [146]:
df_bienImmo[df_bienImmo["nbrePieces"]==1]['surfLot'].mean()

27.104307986468548

In [148]:
df_bienImmo[df_bienImmo["nbrePieces"]==2]['surfLot'].mean()

42.90510598069482

In [149]:
df_bienImmo.loc[16300,'nbrePieces']=2

In [150]:
df_bienImmo[df_bienImmo["nbrePieces"]==10]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
12929,2,350.13,10,Maison,291,RUE,NESTOR LONGUE EPEE,59235,BERSEE,59,2020-06-29,433202.0,Vente
17468,26,197.00,10,Appartement,15,RUE,DES ANEMONES,77400,LAGNY-SUR-MARNE,77,2020-03-03,415000.0,Vente
22795,1,276.57,10,Maison,22,RUE,DE LA LIBERATION,91150,MORIGNY-CHAMPIGNY,91,2020-03-24,330000.0,Vente
34117,55,15.35,10,Appartement,10,RUE,DE SEVIGNE,75004,PARIS 04,75,2020-05-29,242000.0,Vente


The last one is not right. It must be 0 for nbrePieces

In [151]:
df_bienImmo.loc[34117,'nbrePieces']=0

In [152]:
df_bienImmo[df_bienImmo["nbrePieces"]==9]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
6795,1,212.88,9,Maison,25,RUE,DE NUITS,33100,BORDEAUX,33,2020-01-21,780000.0,Vente
8855,11,250.29,9,Appartement,4,PL,VERDUN,38000,GRENOBLE,38,2020-06-12,706500.0,Vente
10160,11,558.87,9,Appartement,18,RUE,PAUL BELLAMY,44000,NANTES,44,2020-05-20,1608880.0,Vente
11053,1,80.00,9,Maison,1,,BOIS DE LA CAVE HAUTE,45320,SAINT-HILAIRE-LES-ANDRESIS,45,2020-06-11,198000.0,Vente
15062,92,92.60,9,Appartement,294,RTE,DE GENAS,69500,BRON,69,2020-06-26,164000.0,Vente
15836,10,130.96,9,Appartement,6,FG,DES ANNONCIADES,74000,ANNECY,74,2020-01-31,599000.0,Vente
16402,1,137.41,9,Maison,173,RUE,MAL JOFFRE,76600,LE HAVRE,76,2020-01-30,144000.0,Vente
18402,1,252.79,9,Maison,9,RTE,DE LA PLAINE,78110,VESINET (LE),78,2020-02-28,2121150.0,Vente
20142,13,104.04,9,Appartement,35,RUE,SUFFREN,83000,TOULON,83,2020-05-26,595000.0,Vente
24089,44,20.25,9,Appartement,11,RUE,JEAN JAURES,92300,LEVALLOIS-PERRET,92,2020-05-22,184762.0,Vente


For the rows 15062, 11053, 15836,20142 there are little surfaces with probably rooms between 3 and 5

In [155]:
df_bienImmo.loc[[15062,11053,15836,20142],'nbrePieces']=4

For 24089, it's very probably 1 room

In [156]:
df_bienImmo.loc[24089,'nbrePieces']=1

In [157]:
df_bienImmo[df_bienImmo["nbrePieces"]==8]

,numLot,surfLot,nbrePieces,typLoc,numVoie,typVoie,voie,cp,ville,codeDep,datMut,valFonc,natMut
441,2,245.13,8,Maison,9005,AV,DES SAVELS,4100,MANOSQUE,4,2020-06-18,300000.0,Vente
2174,36,214.88,8,Appartement,30,RUE,JAILLANT DESCHAINETS,10000,TROYES,10,2020-05-22,324000.0,Vente
8831,24,238.18,8,Appartement,8,RUE,HECTOR BERLIOZ,38000,GRENOBLE,38,2020-05-18,726700.0,Vente
9003,7,136.31,8,Maison,7,ALL,DU CLOS DU CHATEAU,38640,CLAIX,38,2020-06-23,533500.0,Vente
9129,3,67.44,8,Maison,2057,PL,DES LANDAIS,40150,SOORTS-HOSSEGOR,40,2020-01-11,280000.0,Vente
13204,3,220.72,8,Maison,59,RUE,DU GENERAL MANGIN,60200,COMPIEGNE,60,2020-01-03,594700.0,Vente
14353,31,320.92,8,Appartement,5,PL,ANTONIN PONCET,69002,LYON 2EME,69,2020-05-15,2072500.0,Vente
16644,31,210.92,8,Maison,31,ALL,DE LA COMTESSE DE NOAILLES,77820,CHATELET-EN-BRIE (LE),77,2020-01-24,329000.0,Vente
16719,48,290.00,8,Maison,6,RES,LA CHARBONNIERE,77630,BARBIZON,77,2020-02-13,1250000.0,Vente
18276,34,163.19,8,Maison,4,RUE,DES PICOTEUSES,78350,JOUY EN JOSAS,78,2020-06-26,661600.0,Vente
